In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import json

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import torch
from gradslam.structures.pointclouds import Pointclouds
from PIL import Image

from core.dataset import ScanNet
from core.integrate import FeatureFusionScalableTSDFVolume
from core.labeler import CLIPTextQuerier, KMeansLabeler

In [3]:
o3d._build_config["ENABLE_HEADLESS_RENDERING"]

True

In [4]:
scene = "scannet_scene0011_00"
load_dir = "/storage/quanta/Experiments/feature-instance-fusion/" + scene + "/"

In [5]:
dataset = ScanNet('/home/quanta/Datasets/ScanNet/')
nyu40_color = dataset.nyu40id_to_color
nyu40_class = [dataset.nyu40_id_to_class[i] for i in range(41)]

# 01 NYU40 of gradslam conceptfusion

In [6]:
pcd = Pointclouds.load_pointcloud_from_h5(load_dir + 'gradslam_feature_conceptfusion')

In [7]:
feats = pcd.embeddings_padded[0]

In [8]:
pcd_o3d = pcd.open3d(0)

In [9]:
json_pth = os.path.abspath("../../config/views/" + scene + ".json")
with open(json_pth, "r") as f:
    views = json.load(f)['views']

In [10]:
save_dir = "../02_conceptfusion/01_nyu40_3d_query/00_gradslam_color/"
os.makedirs(save_dir, exist_ok=True)

In [11]:
for i, view in enumerate(views):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd_o3d)
    vis.set_view_status(json.dumps(view["view"]))
    buffer = vis.capture_screen_float_buffer(do_render=True)
    image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
    # display(image)
    image.save(save_dir + "{:02d}_".format(i) + view["tag"] + ".png")

In [12]:
clip_querier = CLIPTextQuerier(device='cuda:1')
clip_querier.load_model()

In [13]:
labels = clip_querier.multi_text_query(
    texts=nyu40_class,
    img_feats=feats,
)

In [14]:
pcd_o3d.colors = o3d.utility.Vector3dVector(nyu40_color[labels] / 255)

In [15]:
save_dir = "../02_conceptfusion/01_nyu40_3d_query/01_gradslam_conceptfusion/"
os.makedirs(save_dir, exist_ok=True)

In [16]:
for i, view in enumerate(views):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd_o3d)
    vis.set_view_status(json.dumps(view["view"]))
    buffer = vis.capture_screen_float_buffer(do_render=True)
    image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
    # display(image)
    image.save(save_dir + "{:02d}_".format(i) + view["tag"] + ".png")

# 02 TSDF conceptfusion

In [17]:
tsdf_device = "cuda:1"
tsdf_volume = FeatureFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device="cuda:1",
)
tsdf_volume.load(load_dir + "tsdf/tsdf_volume.pt")
verts = np.load(load_dir + "tsdf/verts.npy")
faces = np.load(load_dir + "tsdf/faces.npy")
tsdf_volume.reset_feature(dim=512, include_var=False, dtype=torch.float32)
tsdf_volume.load_feats(load_dir + "tsdf_feature_conceptfusion/feats.pt")

In [18]:
verts_feats = tsdf_volume.extract_feat_on_grid(verts=verts, device='cpu')[0]

In [19]:
labels = clip_querier.multi_text_query(
    texts=nyu40_class,
    img_feats=torch.from_numpy(verts_feats),
)

In [20]:
mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(verts),
    triangles=o3d.utility.Vector3iVector(faces),
)
color = nyu40_color[labels] / 255
mesh.vertex_colors = o3d.utility.Vector3dVector(color)

In [21]:
save_dir = "../02_conceptfusion/01_nyu40_3d_query/02_tsdf_conceptfusion/"
os.makedirs(save_dir, exist_ok=True)

In [22]:
for i, view in enumerate(views):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(mesh)
    vis.set_view_status(json.dumps(view["view"]))
    buffer = vis.capture_screen_float_buffer(do_render=True)
    image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
    # display(image)
    image.save(save_dir + "{:02d}_".format(i) + view["tag"] + ".png")

# 03 TSDF Conceft fusion Example

In [23]:
test_sents = [
    "There are two ovens under the kitchen counter. The counter is directly beneath a window, adjoining the wall to which a television is mounted.",
    "A stainless steel refrigerator by the dining table and the kitchen counter. The refrigerator is just beside the kitchen sink.",
    "Television",
]

In [24]:
for i, sent in enumerate(test_sents):
    scores = clip_querier.single_text_score(
        text=sent,
        img_feats=torch.from_numpy(verts_feats),
    )
    scores = np.clip(scores, 0.15, 1.0)
    # plt.hist(scores, bins=100)[2]
    color = clip_querier.score_to_color(
        score=scores,
        scheme="jet",
        normalize=True,
        threshold=0.5,
    )

    mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(verts),
        triangles=o3d.utility.Vector3iVector(faces),
    )
    mesh.vertex_colors = o3d.utility.Vector3dVector(color)

    save_dir = "../02_conceptfusion/02_single_word_similarity_plot/" + "{:02d}".format(i) + "/"
    os.makedirs(save_dir, exist_ok=True)

    for j, view in enumerate(views):
        vis = o3d.visualization.Visualizer()
        vis.create_window()
        vis.add_geometry(mesh)
        vis.set_view_status(json.dumps(view["view"]))
        buffer = vis.capture_screen_float_buffer(do_render=True)
        image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
        # display(image)
        image.save(save_dir + "{:02d}_".format(j) + view["tag"] + ".png")

    with open(save_dir + "query.txt", "w") as f:
        f.write(sent)

# 04 tsdf laseg

In [25]:
tsdf_device = "cuda:1"
tsdf_volume = FeatureFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device="cuda:1",
)
tsdf_volume.load(load_dir + "tsdf/tsdf_volume.pt")
verts = np.load(load_dir + "tsdf/verts.npy")
faces = np.load(load_dir + "tsdf/faces.npy")
tsdf_volume.reset_feature(dim=512, include_var=False, dtype=torch.float32)
tsdf_volume.load_feats(load_dir + "tsdf_feature_lseg/feats.pt")

In [26]:
verts_feats = tsdf_volume.extract_feat_on_grid(verts=verts, device='cpu')[0]

In [27]:
labels = clip_querier.multi_text_query(
    texts=nyu40_class,
    img_feats=torch.from_numpy(verts_feats),
)

In [80]:
mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(verts),
    triangles=o3d.utility.Vector3iVector(faces),
)
color = nyu40_color[labels] / 255
mesh.vertex_colors = o3d.utility.Vector3dVector(color)

In [83]:
save_dir = "../02_conceptfusion/01_nyu40_3d_query/03_tsdf_lseg/"
os.makedirs(save_dir, exist_ok=True)

In [84]:
for i, view in enumerate(views):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(mesh)
    vis.set_view_status(json.dumps(view["view"]))
    buffer = vis.capture_screen_float_buffer(do_render=True)
    image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
    # display(image)
    image.save(save_dir + "{:02d}_".format(i) + view["tag"] + ".png")